### Object Table Metric and ccdVisit Table differences between "HSC/runs/RC2/w_2024_38/DM-46429" and "u/yusra/RC2/b7289/DM-43077"

In [1]:
from lsst.daf.butler import Butler
import pandas as pd

In [2]:
butler = Butler("/repo/main", collections="HSC/runs/RC2/w_2024_38/DM-46429")

In [3]:
df = butler.get("objectTableCore_metricsTable", skymap="hsc_rings_v1")

In [4]:
butler_ci = Butler("/repo/main", collections="u/yusra/RC2/b7289/DM-43077")
new = butler_ci.get("objectTableCore_metricsTable", skymap="hsc_rings_v1")

In [5]:
# remove all columns that have arrays:
names = [name for name in df.colnames if len(df[name].shape) <= 1]

df = df[names].to_pandas()
new = new[names].to_pandas()

pd.set_option('display.max_columns', None)
df.compare(new, align_axis=1)


blendMetrics_numBlends           blendMetrics_meanBlendIterations  \
                    self     other                             self   
0               265705.0  277856.0                        17.609992   
1               254619.0  266385.0                        17.803777   
2               229711.0  223817.0                        17.414086   

             blendMetrics_meanBlendLogL                \
       other                       self         other   
0  17.515969              -20479.648438 -1.969536e+04   
1  17.727515             -151156.781250 -9.294938e+11   
2  17.353838              -78580.140625 -7.718287e+04   

  coaddInputCount_g_inputCount_median       coaddInputCount_g_inputCount_mean  \
                                 self other                              self   
0                                 NaN   NaN                          4.373697   
1                                 NaN   NaN                          4.875758   
2                                14.0  13.0                         12.948652   

             coaddInputCount_r_inputCount_mean             \
       other                              self      other   
0   4.393239                          4.364712   4.363734   
1   4.829974                          4.896800   4.887196   
2  12.077048                         13.274271  13.232956   

  coaddInputCount_i_inputCount_mean             \
                               self      other   
0                          6.519592   6.529870   
1                          6.931949   6.891765   
2                         25.554370  25.392911   

  coaddInputCount_z_inputCount_mean             \
                               self      other   
0                          6.499136   6.495013   
1                          7.347639   7.321634   
2                         24.495097  24.389446   

  coaddInputCount_y_inputCount_mean            e1Diff_g_highSNStars_median  \
                               self      other                        self   
0                          6.464847   6.432677                    0.002137   
1                          7.314961   7.251944                    0.013204   
2                         40.466093  40.083941                    0.000567   

            e1Diff_g_highSNStars_sigmaMad            \
      other                          self     other   
0  0.000072                      0.004596  0.001730   
1  0.010113                      0.006423  0.009488   
2  0.000821                      0.003195  0.003255   

  e1Diff_g_highSNStars_count        e1Diff_g_lowSNStars_median            \
                        self  other                       self     other   
0                       11.0    8.0                   0.000213  0.000235   
1                       19.0   17.0                   0.000155  0.000270   
2                      793.0  663.0                   0.000008  0.000201   

  e1Diff_g_lowSNStars_sigmaMad           e1Diff_g_lowSNStars_count          \
                          self     other                      self   other   
0                     0.003939  0.003996                    4841.0  4759.0   
1                     0.004236  0.004417                    3268.0  3217.0   
2                     0.003895  0.003979                    5123.0  4594.0   

  e1Diff_r_highSNStars_median           e1Diff_r_highSNStars_sigmaMad  \
                         self     other                          self   
0                         NaN       NaN                           NaN   
1                    0.004006  0.005466                      0.001127   
2                    0.000717 -0.000070                      0.003168   

            e1Diff_r_highSNStars_count        e1Diff_r_lowSNStars_median  \
      other                       self  other                       self   
0       NaN                        NaN    NaN                  -0.000118   
1  0.001178                        4.0    3.0                   0.000542   
2  0.003702                      732.

In [6]:
import pandas as pd

def compare_dataframes(df1, df2, key_column='tract'):
    merged_df = pd.merge(df1, df2, on=key_column, suffixes=('_df1', '_df2'))
    
    # Initialize a list to store the comparison results
    comparison_results = []

    # Iterate over the columns in df1 and df2, except the key column
    for col in df1.columns:
        if col == key_column:
            continue  # Skip the key column

        col_df1 = col + '_df1'
        col_df2 = col + '_df2'

        # Compare values for each column and store results
        merged_df['match'] = merged_df[col_df1] == merged_df[col_df2]
        
        differences = merged_df[~merged_df['match']][[key_column, col_df1, col_df2]]
        
        for _, row in differences.iterrows():
            comparison_results.append({
                 key_column: row[key_column],
                'column': col,
                'main': row[col_df1],
                'DM-43077': row[col_df2],
                'match': False,      
            })
            
    comparison_df = pd.DataFrame(comparison_results)
    return comparison_df

In [7]:
result = compare_dataframes(df, new)
result["difference"] = result["DM-43077"] - result["main"]
result["frac_diff"] = result["difference"]/result["main"]

In [8]:
pd.set_option('display.float_format', '{:.4f}'.format)
print("Every metric that increased by >50%")
result[result["frac_diff"] > 1.5][["tract", "column", "main", "DM-43077", "difference", "frac_diff"]].sort_values("frac_diff", ascending=False)

Every metric that increased by >50%


,tract,column,main,DM-43077,difference,frac_diff
7,9697.0000,blendMetrics_meanBlendLogL,-151156.7812,-929493811200.0000,-929493660043.2188,6149202.5191
141,9813.0000,e2Diff_r_lowSNStars_median,0.0000,0.0012,0.0012,159.3879
71,9813.0000,e1Diff_i_lowSNStars_median,0.0000,0.0011,0.0011,57.0459
36,9813.0000,e1Diff_g_lowSNStars_median,0.0000,0.0002,0.0002,25.3245
382,9813.0000,skyFluxStatisticMetric_r_meanSky,-0.8280,-20.5945,-19.7665,23.8726
305,9813.0000,shapeSizeFractionalDiff_i_highSNStars_median,0.0004,0.0066,0.0062,17.0738
332,9813.0000,shapeSizeFractionalDiff_z_lowSNStars_median,0.0007,0.0088,0.0081,12.3669
341,9813.0000,shapeSizeFractionalDiff_y_highSNStars_median,0.0007,0.0092,0.0084,11.4142
385,9813.0000,skyFluxStatisticMetric_r_stdevSky,122.7357,1071.6672,948.9315,7.7315
349,9813.0000,shapeSizeFractionalDiff_y_lowSNStars_median,0.0013,0.0095,0.0082,6.2808


In [9]:
import numpy as np
result[np.isnan(result["frac_diff"])][["tract", "column", "main", "DM-43077", "frac_diff"]].sort_values("frac_diff", ascending=False)

,tract,column,main,DM-43077,frac_diff
43,9615.0000,e1Diff_r_highSNStars_median,NaN,NaN,NaN
46,9615.0000,e1Diff_r_highSNStars_sigmaMad,NaN,NaN,NaN
97,9697.0000,e1Diff_y_highSNStars_median,NaN,NaN,NaN
100,9697.0000,e1Diff_y_highSNStars_sigmaMad,NaN,NaN,NaN
131,9615.0000,e2Diff_r_highSNStars_median,NaN,NaN,NaN
134,9615.0000,e2Diff_r_highSNStars_sigmaMad,NaN,NaN,NaN
185,9697.0000,e2Diff_y_highSNStars_median,NaN,NaN,NaN
188,9697.0000,e2Diff_y_highSNStars_sigmaMad,NaN,NaN,NaN
286,9615.0000,shapeSizeFractionalDiff_r_highSNStars_median,NaN,NaN,NaN
289,9615.0000,shapeSizeFractionalDiff_r_highSNStars_sigmaMad,NaN,NaN,NaN


In [10]:
ccdVisit = butler_ci.get("ccdVisitTable", skymap="hsc_rings_v1")
ccdVisit_main = butler.get("ccdVisitTable", skymap="hsc_rings_v1")

In [11]:
ccdVisit.columns

Index(['visitId', 'physical_filter', 'band', 'ra', 'dec', 'pixelScale',
       'zenithDistance', 'expTime', 'zeroPoint', 'psfSigma', 'skyBg',
       'skyNoise', 'astromOffsetMean', 'astromOffsetStd', 'nPsfStar',
       'psfStarDeltaE1Median', 'psfStarDeltaE2Median', 'psfStarDeltaE1Scatter',
       'psfStarDeltaE2Scatter', 'psfStarDeltaSizeMedian',
       'psfStarDeltaSizeScatter', 'psfStarScaledDeltaSizeScatter',
       'psfTraceRadiusDelta', 'psfApFluxDelta', 'psfApCorrSigmaScaledDelta',
       'maxDistToNearestPsf', 'effTime', 'effTimePsfSigmaScale',
       'effTimeSkyBgScale', 'effTimeZeroPointScale', 'magLim', 'decl',
       'detector', 'seeing', 'skyRotation', 'expMidpt', 'expMidptMJD',
       'obsStart', 'obsStartMJD', 'darkTime', 'xSize', 'ySize', 'llcra',
       'llcdec', 'ulcra', 'ulcdec', 'urcra', 'urcdec', 'lrcra', 'lrcdec'],
      dtype='object')

In [12]:
ccd_diff = compare_dataframes(ccdVisit_main.reset_index(), ccdVisit.reset_index(),  key_column='ccdVisitId')

In [13]:
ccd_diff['diff'] =  ccd_diff['DM-43077'] - ccd_diff['main']
ccd_diff['frac_diff'] = ccd_diff['diff']/ccd_diff['main']

In [14]:
ccd_diff[ccd_diff['column'] == "visitId"] 

,ccdVisitId,column,main,DM-43077,match,diff,frac_diff


In [15]:
ccd_diff[(ccd_diff['frac_diff'] > 1.5) & (~ccd_diff.column.isin(("skyNoise", "skyBg", "effTimeSkyBgScale")))]

,ccdVisitId,column,main,DM-43077,match,diff,frac_diff
202686,7236480.0000,psfSigma,1.1026,3.6276,False,2.5250,2.2900
209440,7362036.0000,psfSigma,1.2686,3.6554,False,2.3868,1.8813
314086,4782034.0000,astromOffsetMean,0.1236,0.3106,False,0.1870,1.5123
414700,63602.0000,psfStarDeltaE1Median,0.0002,0.0007,False,0.0005,3.3397
414705,63607.0000,psfStarDeltaE1Median,0.0007,0.0029,False,0.0022,3.0747
...,...,...,...,...,...,...,...
946579,7243243.0000,effTimePsfSigmaScale,0.4065,1.7849,False,1.3784,3.3907
946881,7247236.0000,effTimePsfSigmaScale,1.9023,27.3612,False,25.4589,13.3831
1111931,7236480.0000,seeing,0.4310,1.4181,False,0.9870,2.2900
1118685,7362036.0000,seeing,0.4880,1.4061,False,0.9181,1.8813


In [16]:
ccd_diff[(ccd_diff['frac_diff'] < -1.5) & (~ccd_diff.column.isin(("skyNoise", "skyBg", "effTimeSkyBgScale")))]

,ccdVisitId,column,main,DM-43077,match,diff,frac_diff
414701,63603.0000,psfStarDeltaE1Median,-0.0002,0.0015,False,0.0017,-7.5482
414702,63604.0000,psfStarDeltaE1Median,-0.0014,0.0008,False,0.0022,-1.5798
414707,63610.0000,psfStarDeltaE1Median,-0.0004,0.0011,False,0.0015,-4.2194
414709,63612.0000,psfStarDeltaE1Median,-0.0008,0.0006,False,0.0013,-1.7375
414712,63615.0000,psfStarDeltaE1Median,-0.0003,0.0002,False,0.0005,-1.5687
...,...,...,...,...,...,...,...
622134,7788899.0000,psfStarDeltaSizeMedian,-0.0000,0.0006,False,0.0006,-14.2764
622135,7788900.0000,psfStarDeltaSizeMedian,0.0003,-0.0009,False,-0.0012,-3.9391
622138,7788903.0000,psfStarDeltaSizeMedian,-0.0004,0.0010,False,0.0014,-3.4084
622173,7790035.0000,psfStarDeltaSizeMedian,-0.0003,0.0008,False,0.0012,-3.4946


In [17]:
ccd_diff[(ccd_diff['frac_diff'] > 1.5) & (~ccd_diff.column.isin(("skyNoise", "skyBg", "effTimeSkyBgScale")))].groupby("column").median()[["main","DM-43077","frac_diff"]]

,main,DM-43077,frac_diff
column,,,
astromOffsetMean,0.1236,0.3106,1.5123
effTime,270.5785,1117.4636,2.4834
effTimePsfSigmaScale,1.6821,27.3612,13.3831
maxDistToNearestPsf,583.0497,1658.7123,2.0380
psfApCorrSigmaScaledDelta,0.0136,0.0477,2.2141
psfApFluxDelta,0.0230,0.0721,2.0256
psfSigma,1.1856,3.6415,2.0857
psfStarDeltaE1Median,-0.0000,-0.0002,3.4341
psfStarDeltaE1Scatter,0.0069,0.0199,1.7675


In [18]:
ccd_diff[(ccd_diff['frac_diff'] > 1.5) & (~ccd_diff.column.isin(("skyNoise", "skyBg", "effTimeSkyBgScale")))].groupby("column").count().ccdVisitId

column
astromOffsetMean                     1
effTime                          21904
effTimePsfSigmaScale                 3
maxDistToNearestPsf                  6
psfApCorrSigmaScaledDelta         1119
psfApFluxDelta                     691
psfSigma                             2
psfStarDeltaE1Median              4743
psfStarDeltaE1Scatter               13
psfStarDeltaE2Median              4865
psfStarDeltaE2Scatter               11
psfStarDeltaSizeMedian            4654
psfStarDeltaSizeScatter              9
psfStarScaledDeltaSizeScatter        9
psfTraceRadiusDelta                 94
seeing                               2
ulcdec                               1
Name: ccdVisitId, dtype: int64

In [19]:
ccd_diff[(ccd_diff['frac_diff'] < -1.5) & (~ccd_diff.column.isin(("skyNoise", "skyBg", "effTimeSkyBgScale")))].groupby("column").count().ccdVisitId

column
psfStarDeltaE1Median      6902
psfStarDeltaE2Median      7064
psfStarDeltaSizeMedian    6407
Name: ccdVisitId, dtype: int64